In [0]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.optimizers import SGD
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [0]:
data = pd.read_pickle('drive/My Drive/treject/mlp_feed.pkl')  
clusters = np.loadtxt('drive/My Drive/treject/cluster_centers.txt')
# see what final features look like
data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,DAY_TYPE,MISSING_DATA,POLYLINE,RIDE_DURATION,TIME,WEEK_OF_YEAR,DAY_OF_WEEK,QUARTER_HOUR,LAT_1,LAT_2,LAT_3,LAT_4,LAT_5,LAT_6,LAT_7,LAT_8,LAT_9,LAT_10,LNG_1,LNG_2,LNG_3,LNG_4,LNG_5,LNG_6,LNG_7,LNG_8,LNG_9,LNG_10,TARGET_LAT,TARGET_LNG
0,1372636858620000589,2,0,0,366,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",345,2013-07-01 00:00:58,27,0,0.0,-8.618643,-8.618499,-8.620326,-8.622153,-8.623953,-8.632323,-8.631144,-8.630829,-8.630829,-8.630829,41.141412,41.141376,41.142510,41.143815,41.144373,41.153022,41.154489,41.154507,41.154516,41.154498,-8.630838,41.154489
1,1372637303620000596,1,0,7,370,0,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",285,2013-07-01 00:08:23,27,0,0.0,-8.639847,-8.640351,-8.642196,-8.644455,-8.646921,-8.670942,-8.669610,-8.668098,-8.666640,-8.665767,41.159826,41.159871,41.160114,41.160492,41.160951,41.166576,41.167962,41.168988,41.170005,41.170635,-8.665740,41.170671
2,1372636951620000320,2,0,0,203,0,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",975,2013-07-01 00:02:31,27,0,0.0,-8.612964,-8.613378,-8.614215,-8.614773,-8.615907,-8.625996,-8.624088,-8.621325,-8.619444,-8.617365,41.140359,41.140350,41.140278,41.140368,41.140449,41.144769,41.144463,41.143401,41.141961,41.140863,-8.615970,41.140530
3,1372636854620000520,2,0,0,329,0,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",645,2013-07-01 00:00:54,27,0,0.0,-8.574678,-8.574705,-8.574696,-8.574660,-8.574723,-8.607951,-8.607978,-8.607996,-8.607987,-8.608005,41.151951,41.151942,41.151933,41.151960,41.151933,41.142753,41.142825,41.142879,41.142888,41.142915,-8.607996,41.142915
4,1372637091620000337,2,0,0,216,0,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",435,2013-07-01 00:04:51,27,0,0.0,-8.645994,-8.645949,-8.646048,-8.646804,-8.649495,-8.689086,-8.689059,-8.687502,-8.687259,-8.687259,41.180490,41.180517,41.180049,41.178888,41.178465,41.176431,41.176593,41.176755,41.177781,41.178078,-8.687268,41.178087


getting the data into the necessary format to feed to the neural network. Creating the Numpy Arrays to feed into the model

In [0]:
caller_data = np.array(data.ORIGIN_CALL)
stand_data = np.array(data.ORIGIN_STAND)
car_data = np.array(data.TAXI_ID)
wk_of_year_data = np.array(data.WEEK_OF_YEAR)
day_of_wk_data = np.array(data.DAY_OF_WEEK)
qrt_hr_of_day_data = np.array(data.QUARTER_HOUR)
day_type_data = np.array(data.DAY_TYPE)

target_lat = np.array(data.TARGET_LAT)
target_lng = np.array(data.TARGET_LNG)

lat_data = []
lng_data = []

for i in range(len(data)):
  
  lat_temp = np.array([data['LAT_'+ str(j)][i] for j in range(1,11)])
  lng_temp = np.array([data['LNG_'+ str(j)][i] for j in range(1,11)])
  lat_data.append(lat_temp)
  lng_data.append(lng_temp)

In [0]:
target = np.array([np.array([i,j]) for i, j in zip(target_lat, target_lng)])
lat_data = np.array(lat_data)
lng_data = np.array(lng_data)
lat_ = lat_data
lng_ = lng_data
lat_data = scale(lat_data, axis=0, with_mean=True, with_std=True, copy=True)
lng_data = scale(lng_data, axis=0, with_mean=True, with_std=True, copy=True)

Helper functions that set up the 'destination layer' and the loss function we will use using the Tensorflow backend.

In [0]:
def tf_haversine(latlon1, latlon2):
    """
    Tensorflow version of the Haversine function to calculate distances between two sets of points.
    """
    lat1 = latlon1[:, 0]
    lon1 = latlon1[:, 1]
    lat2 = latlon2[:, 0]
    lon2 = latlon2[:, 1]

    REarth = 6371
    lat = tf.abs(lat1 - lat2) * np.pi / 180
    lon = tf.abs(lon1 - lon2) * np.pi / 180
    lat1 = lat1 * np.pi / 180
    lat2 = lat2 * np.pi / 180
    a = tf.sin(lat / 2) * tf.sin(lat / 2) + tf.cos(lat1) * tf.cos(lat2) * tf.sin(lon / 2) * tf.sin(lon / 2)
    d = 2 * tf.atan2(tf.sqrt(a), tf.sqrt(1 - a))
    return REarth * d
  
def destination(probabilities):
  return tf.matmul(probabilities, cast_clusters) # Cast clusters will be a constant matrix of (Nx2)

define input layers

In [0]:
caller = Input(shape=(1,))
stand  = Input(shape=(1,))
car  = Input(shape=(1,))
wk_of_year = Input(shape=(1,))
day_of_wk = Input(shape=(1,))
qrt_hr_of_day = Input(shape=(1,))
day_type = Input(shape=(1,))
lat = Input(shape=(10,))
lng = Input(shape=(10,))
trg = Input(shape=(2,))

W0806 10:21:10.190352 140341230536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 10:21:10.208029 140341230536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



creating the embedding layer for the metadata, the embedding layer outputs are vectors of 10.

In [0]:
caller_embed = Embedding(57000,10,input_length=1)(caller)
stand_embed = Embedding(64,10,input_length=1)(stand)
car_embed = Embedding(448,10,input_length=1)(car)
wk_of_year_embed = Embedding(54,10,input_length=1)(wk_of_year)
day_of_wk_embed = Embedding(7,10,input_length=1)(day_of_wk)
qrt_hr_of_day_embed = Embedding(96,10,input_length=1)(qrt_hr_of_day)
day_type_embed = Embedding(3,10,input_length=1)(day_type)

# flatten
caller_embed = Flatten()(caller_embed)
stand_embed = Flatten()(stand_embed)
car_embed = Flatten()(car_embed)
wk_of_year_embed = Flatten()(wk_of_year_embed)
day_of_wk_embed = Flatten()(day_of_wk_embed)
qrt_hr_of_day_embed = Flatten()(qrt_hr_of_day_embed)
day_type_embed = Flatten()(day_type_embed)

# pass lat lng through own network
postional_features = Concatenate(axis=-1)([lat,lng])
positional = Dense(1, input_dim=20)(postional_features)

W0806 10:21:10.230579 140341230536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



merge layers

In [0]:
input_features = [caller_embed, stand_embed, car_embed, wk_of_year_embed, day_of_wk_embed, qrt_hr_of_day_embed, day_type_embed, positional]
input_features = Concatenate(axis=-1)(input_features)

create hidden and classification layer

In [0]:
DNN = Dense(512, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',
                         kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                         kernel_constraint=None, bias_constraint=None)(input_features)
classification = Dense(len(clusters), activation='softmax')(DNN)

After getting the predictions , taking the weighted average of the cluster centers.

In [0]:
cast_clusters = K.cast_to_floatx(clusters)
destination_layer = Activation(destination)(classification)
# gather model together
model = Model(inputs=[caller, stand, car, wk_of_year, day_of_wk, qrt_hr_of_day, day_type, lat, lng], 
              outputs=destination_layer)

Defining the SGD optimizer & compiling the model

In [0]:
# optimizer 
optimizer = SGD(lr=0.01, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients
model.compile(loss=tf_haversine, optimizer=optimizer)

W0806 10:21:10.433631 140341230536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# take a look at the model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (I

Random guess averages 6-7 Km difference with ground truth.

Fitting the model..

In [0]:
history = model.fit([caller_data, stand_data, car_data,
                     wk_of_year_data, day_of_wk_data, qrt_hr_of_day_data,
                     day_type_data, lat_data, lng_data], 
                     y=target, validation_split=0.2,batch_size = 200, epochs=5)